<a href="https://colab.research.google.com/github/rosacarla/Web-scraping-with-Python/blob/main/summarization_mainsubject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> # SUMARIZAÇÃO E TEMAS CENTRAIS
CURSO: Tecnólogo em Inteligência Artificial Aplicada  
DISCIPLINA: Agentes Conversacionais  
AUTOR: Carla Edila Santos da Rosa Silveira  
REQUISITO: construir algoritmo para sumarizar notícias e reportagens de um portal e encontrar assunto principal do texto buscado.  
DATA: 04/09/2023


-----------------------------------------------------------------------  
<body>
<center>
<img src="https://i.postimg.cc/RZbnDjsz/summarization-image.jpg" align="middle">
</body>

> ## 1. CONTEXTO  
> <p align="justify">A demanda é desenvolver um algoritmo que leia automaticamente notícias ou reportagens de determinado portal e apresente os principais assuntos relacionados a cada texto.</p>  
> <p align="justify">A intenção é que o usuário informe uma palavra-chave para o algoritmo buscar todas os textos que contenham esse termo no título e apresente um resumo dele com as principais palavras ligadas ao conteúdo.</p>



> ## 2. COMO RESOLVER
> <p align="justify">Será utilizada a biblioteca <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc.ptbr/#:~:text=Beautiful%20Soup%20%C3%A9%20uma%20biblioteca,de%20an%C3%A1lise%20(parse%20tree)">Beautiful Soup</a> para fazer o web-scraping dos dados do portal informativo, pré-processar os textos, gerar a bag of words, usar uma metodologia de sumarização baseada em frequência para selecionar as sentenças mais importantes.</p>  


> ## 3. Importa bibliotecas necessárias ao WEB-SCRAPING e demais atividades

In [118]:
import requests
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import collections
from string import punctuation
from heapq import nlargest
import nltk
from nltk import tokenize
from nltk import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

> ## 4. Pergunta ao usuário qual é a palavra-chave

In [119]:

keyword = input("Informe a palavra-chave para a busca: ")

Informe a palavra-chave para a busca: reforma


> **OBSERVAÇÃO:** usar palavra-chave com grafia usual no site, porque o modelo diferencia letras maiúscula e minúsculas.

> ## 5. Função busca links de todos os textos que tenham a palavra-chave dada no título

In [120]:

def obtem_links_relacionados(keyword):

  resp = requests.get("https://www.globo.com/") # Obtem HTML da página
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding) # Interpreta como XML e no encoding correto

  links = []
  # Obtem e percorre os links da pagina principal do portal
  for link in soup.find_all("a", {"class": "post__link"}):
    # Se o titulo do texto tiver a palavra-chave
    if link.get("title").find(keyword) != -1:
      # Imprime o titulo
      print("Encontrei o texto: " + link.get("title"))
      # Guarda link do texto na lista
      links.append( link.get("href") )

  return links

> ## 6. Função obtém os textos a partir do link dado

In [121]:

def obtem_texto(link):

  resp = requests.get(link) # Obtem o HTML da pagina
  http_encoding = resp.encoding if 'charset' in resp.headers.get('content-type', '').lower() else None
  html_encoding = EncodingDetector.find_declared_encoding(resp.content, is_html=True)
  encoding = html_encoding or http_encoding
  soup = BeautifulSoup(resp.content, 'lxml', from_encoding=encoding) # Interpreta como XML e no encoding correto

  texto = ""
  for p in soup.find_all("p", {"class": "content-text__container"}):
    texto += p.get_text()

  return texto

> ## 7. Pré-processamento

In [122]:

def preprocessar(texto):

  # Deixa todo o texto em minusculas
  texto = texto.lower()

  # Tokeniza o texto
  tokens = tokenize.word_tokenize(texto, language='portuguese')

  tokens_limpos = tokens[:]
  # Pre processa o texto -> remove numeros, pontuacoes, stopwords
  stopwords = set( nltk.corpus.stopwords.words('portuguese') + list(punctuation) + list("0123456789") )
  for token in tokens:

    if token in stopwords:
      tokens_limpos.remove(token)

  return tokens_limpos


> ## 8. Função Bag of Words para obter palavras mais frequentes

In [123]:

def bagofwords(tokens):

  # Usa o FreqDist no NLTK; poderia usar o CountVectorizer do scikit-learn
  freq = nltk.FreqDist(tokens)
  # Obtem as 100 palavras mais frequentes
  top_words = freq.most_common(100)

  return freq, top_words

> ## 9. Função do sumarizador de textos

In [124]:

def sumarizar(texto, top_words, freq):

  # Separa o texto original em sentenças
  sentencas = sent_tokenize(texto)

  # Sentencas com maior quantidade de palavras frequentes tem maior pontuacao
  ranking = collections.defaultdict(int)
  for i, sent in enumerate(sentencas):
    for word in tokenize.word_tokenize(sent.lower()):
      if word in freq:
        ranking[i] += freq[word]
    top_sentences = nlargest(4, ranking, ranking.get) # Quantidade de sentencas pode ser editada nesta linha

  # Ordena as sentencas com maior pontuacao
  sorted_sentences = [sentencas[j] for j in sorted(top_sentences)]

  return sorted_sentences

> ## 10. Teste do sumarizador

In [125]:
# Obtem links dos textos relacionados
links = obtem_links_relacionados(keyword)
print("\nLinks: ", links)

# Percorre os links encontrados
for link in links:

  # Obtem o texto da reportagem
  texto = obtem_texto(link)
  print("\nTexto: ", texto)

  # Efetua pre-processamento
  tokens = preprocessar(texto)
  print("\nTokens: ", tokens)

  # Aplica bag-of-words
  freq, top_words = bagofwords(tokens)

  # Sumariza e obtem temas centrais do texto
  sentencas = sumarizar(texto, top_words, freq)
  print("\nSentenças: ", sentencas)

  # Imprime
  print("\nNOTÍCIA: " + link)
  print("SUMÁRIO: ")
  for s in sentencas:
    print(s)

  print("\nTERMOS CENTRAIS: ")
  print(top_words)
  print("--------------\n\n")

Encontrei o texto: Lula encaminha reforma ministerial e conversará com PSB
Encontrei o texto: Duailibi: saiba como ficou o desenho da reforma

Links:  ['https://g1.globo.com/politica/noticia/2023/09/04/minirreforma-ministerial-deve-ser-anunciada-ate-quarta-mas-lula-ainda-vai-conversar-com-o-psb-de-alckmin.ghtml', 'https://g1.globo.com/politica/blog/julia-duailibi/post/2023/09/04/lula-decide-dar-esporte-para-o-pp-e-portos-e-aeroportos-para-o-republicanos-presidente-vai-conversar-com-ana-moser-e-alckmin.ghtml']

Texto:   Governo quer atrair partidos do Centrão para a base aliada e obter mais votos no Congresso  Lula chegou a cogitar desmembrar a pasta do Desenvolvimento Social e a da Indústria, mas voltou atrás diante do desinteresse do Centrão nessa solução  Dificuldade agora é encontrar uma pasta para acomodar o ministro Márcio França  A minirreforma ministerial, que está sendo desenhada pelo presidente Luiz Inácio Lula da Silva, deve ser anunciada até a quarta-feira (6), depois de vár

> ## 11. Melhorias para o projeto
- Incluir conversão para letra minúscula, passar tudo para o singular na etapa de normalização
- Aplicar o TF-IDF para medir a importância das palavras
- Executar Stemming ou lematização no vocabulário, pois a redução das palavras as torna mais adequadas à lista de tokens
- Criar uma wordcloud dos termos
- Colocar os verbos na lista de stopwords para retirá-los todos
- Utilizar outros algoritmos de sumarização, como a bilioteca <a href="https://pypi.org/project/sumy/">Sumy</a>, que tem vários métodos implementados